丟一些測試的代碼

In [39]:
import pandas as pd
import numpy as np
import cv2
import os
from PIL import Image
import math
import matplotlib.pyplot as plt

In [ ]:
def rgb2hsv(color):
    color_list=color.tolist()
    b,g,r=color_list[0],color_list[1],color_list[2]
    r, g, b = r/255.0, g/255.0, b/255.0
    mx, mn = max(r, g, b), min(r, g, b)
    m = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        if g >= b:
            h = ((g-b)/m)*60
        else:
            h = ((g-b)/m)*60 + 360
    elif mx == g:
        h = ((b-r)/m)*60 + 120
    elif mx == b:
        h = ((r-g)/m)*60 + 240
    if mx == 0:
        s = 0
    else:
        s = m/mx
    v = mx
    return h, s, v


資料庫
===

In [7]:
##資料庫
import pandas as pd
import numpy as np

title={
    "id":[],
    'name_En':[],
    'name_Ch':[],
    'color_hist':[]
}

flower_db=pd.DataFrame(title)

flower_name={
    
}

* tree poppy 樹罌粟
* alpine sea holly 海濱刺芹
* balloon flower 桔梗
* bishop of llandaff 大麗花
* black-eyed susan 黑心金光菊
* blanket flower 天人菊
* californian poppy 花菱草
* daisy 雛菊
* giant white arum lily 海芋
* globe thistle 藍刺頭
* iris 鳶尾花
* lilac hibiscus 朱槿
* orange dahlia 大麗花
* osteospermum 藍眼菊
* pink-yellow dahlia
* poinsettia聖誕紅
* purple coneflower 紫錐花
* sunflower 向日葵
* thorn apple 曼陀羅
* tree mallow 花葵


In [8]:
flower_db
#flower_pic\iris\12769.jpeg

,id,name_En,name_Ch,color_hist


In [48]:
def GetId(img):
    name=img.split("/")
    ID=name[-1][:-5]
    return ID

def GetName(img):
    name=img.split("/")
    Name=name[-2]
    return Name

In [49]:
GetName("flower_pic/iris/12769.jpeg")

'iris'

In [5]:
import Flower
#Flower.show_match('C:/flowerpic/jpeg-224x224/val/iris/12769.jpeg','C:/flowerpic/jpeg-224x224/val/iris/16301.jpeg')


邊緣檢測(?)
---


In [32]:
import cv2
import numpy as np
import imutils
img="flower_pic/tree poppy/12795.jpeg"
image=cv2.imread(img)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#高斯慮波
#v2.GaussianBlur（ SRC，ksize，sigmaX [，DST [，sigmaY [，borderType ] ] ] ）

blurred = cv2.GaussianBlur(gray, (7, 7), 0)

#minVal和maxVal
edged = cv2.Canny(blurred, 50, 200)
cv2.imshow("SIFT", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [33]:
( cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#print("I count {} coins in this image".format(len(cnts)))

coins = image.copy()

cv2.drawContours(coins, cnts, -1, (0, 255, 0), 2)
cv2.imshow("SIFT", coins)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
for (i, c) in enumerate(cnts):

    (x, y, w, h) = cv2.boundingRect(c)

    print("Part #{}".format(i + 1))

    coin = image[y:y + h, x:x + w]

    cv2.imshow("part", coin)

    mask = np.zeros(image.shape[:2], dtype = "uint8")

    ((centerX, centerY), radius) = cv2.minEnclosingCircle(c)

    cv2.circle(mask, (int(centerX), int(centerY)), int(radius), 255, -1)

    mask = mask[y:y + h, x:x + w]
    cv2.imshow("masked part",cv2.bitwise_and(coin, coin, mask = mask))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
  

Part #1
Part #2
Part #3
Part #4


ANN
===